# AI Advice V2 End-to-End Notebook

This notebook mirrors the Python utilities inside `src/ai_advice/v2` so the full planning pipeline can be run and debugged interactively.
It intentionally keeps the same helper functions and defaults found in the V2 scripts while making them easy to call from a notebook cell.


In [ ]:
from __future__ import annotations

from pathlib import Path
from datetime import datetime, UTC, timezone
from typing import Any, Dict, List, Optional
import argparse
import json
import math
import os
import re
import sys
import time
import uuid

from jsonschema import Draft202012Validator
from jsonschema.exceptions import ValidationError
from openai import OpenAI
from dotenv import load_dotenv

# Make project imports work from the notebook location
ROOT_DIR = Path.cwd()
sys.path.insert(0, str(ROOT_DIR))
DATA_DIR = ROOT_DIR / "data" / "ai_advice"
RECOMMEND_DIR = ROOT_DIR / "data" / "recommend"

load_dotenv()


## Standardize recommended papers (from `standardize_input.py`)
The helpers below reproduce the CLI script so we can select a recommend file, normalize text fields, and write a `standardize_input_*.json` file for later steps.


In [ ]:
# --- standardize_input helpers ---

def newest_recommend_file(input_dir: Path, pattern: str) -> Path:
    files = sorted(input_dir.glob(pattern), key=lambda p: p.stat().st_mtime, reverse=True)
    if not files:
        raise FileNotFoundError(f"No files match: {input_dir}/{pattern}")
    return files[0]


def normalize_text(s: Optional[str], max_len: int) -> str:
    if not s:
        return ""
    s = re.sub(r"\s+", " ", s).strip()
    return s[:max_len]


def pick_query(r: Dict[str, Any], max_len: int) -> str:
    return normalize_text(r.get("query"), max_len)


def extract_year(r: Dict[str, Any]) -> int:
    ud = (r.get("update_date") or "").strip()
    if len(ud) >= 4 and ud[:4].isdigit():
        return int(ud[:4])
    vers = r.get("versions") or []
    if vers and isinstance(vers, list):
        created = vers[0].get("created", "")
        m = re.search(r"(\d{4})", created or "")
        if m:
            return int(m.group(1))
    return 0


def to_minimal_record(r: Dict[str, Any], max_abs_len: int, max_query_len: int) -> Dict[str, Any]:
    pid = r.get("id") or r.get("_id") or ""
    title = (r.get("title") or "").strip()
    abstract = normalize_text(r.get("abstract") or "", max_abs_len)
    year = extract_year(r)
    cit = int(r.get("citation_count") or 0)

    if r.get("_score") is not None:
        score = float(r["_score"])
    elif r.get("score") is not None:
        score = float(r["score"])
    elif r.get("sim_score") is not None:
        score = float(r["sim_score"])
    else:
        score = 0.0

    authors = r.get("authors") or r.get("authors_parsed")
    categories = r.get("categories")
    query = pick_query(r, max_query_len)

    return {
        "id": pid,
        "title": title,
        "abstract": abstract,
        "year": year,
        "citation_count": cit,
        "score": score,
        "authors": authors,
        "categories": categories,
        "query": query,
    }


def build_selected_papers(
    mode: str = "application",
    max_abstract_chars: int = 3000,
    max_query_chars: int = 200,
    output_dir: Path = DATA_DIR,
    input_dir: Path = RECOMMEND_DIR,
) -> Path:
    pattern = f"recommend_{mode}.json"
    latest = newest_recommend_file(input_dir, pattern)
    seen = set()
    rows: List[Dict[str, Any]] = []

    with latest.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            r = json.loads(line)
            pid = r.get("id") or r.get("_id")
            if not pid or pid in seen:
                continue
            seen.add(pid)
            rows.append(
                to_minimal_record(
                    r,
                    max_abs_len=max_abstract_chars,
                    max_query_len=max_query_chars,
                )
            )

    timestamp = datetime.now(UTC).strftime("%Y-%m-%d_%H%M")
    out_path = output_dir / f"standardize_input_{timestamp}.json"
    output_dir.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as wf:
        for rec in rows:
            wf.write(json.dumps(rec, ensure_ascii=False) + "
")

    print(f"Processed {len(rows)} papers from {latest.name} -> {out_path}")
    return out_path


## Plan schema contract (from `schema_contract.py`)
Generate or validate the JSON schema that constrains the model output.


In [ ]:
# --- schema_contract helpers ---

PLAN_SCHEMA: Dict[str, Any] = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "description": "Structured learning plan generated from selected papers for a given goal.",
    "title": "PaperTrailLearningPlan",
    "type": "object",
    "additionalProperties": False,
    "required": [
        "plan_overview",
        "reading_order",
        "actions",
        "metrics",
        "timeline_weeks",
        "risks",
        "goal",
        "study_level",
        "source_papers",
        "metadata",
    ],
    "properties": {
        "goal": {"type": "string", "minLength": 1, "maxLength": 2000},
        "study_level": {"type": "string", "enum": ["beginner", "intermediate", "advanced"]},
        "source_papers": {
            "type": "array",
            "minItems": 1,
            "maxItems": 200,
            "items": {"type": "string", "minLength": 1},
        },
        "metadata": {
            "type": "object",
            "additionalProperties": False,
            "required": ["prompt_version", "model", "created_at"],
            "properties": {
                "prompt_version": {"type": "string", "minLength": 1, "maxLength": 100},
                "model": {"type": "string", "minLength": 1, "maxLength": 100},
                "created_at": {"type": "string", "format": "date-time"},
            },
        },
        "plan_overview": {"type": "string", "minLength": 1, "maxLength": 5000},
        "reading_order": {
            "type": "array",
            "minItems": 1,
            "maxItems": 50,
            "items": {
                "type": "object",
                "additionalProperties": False,
                "required": ["paper_id", "why_first", "key_questions"],
                "properties": {
                    "paper_id": {"type": "string", "minLength": 1},
                    "why_first": {"type": "string", "minLength": 1, "maxLength": 2000},
                    "key_questions": {
                        "type": "array",
                        "minItems": 1,
                        "maxItems": 6,
                        "items": {"type": "string", "minLength": 1, "maxLength": 500},
                    },
                },
            },
        },
        "actions": {
            "type": "array",
            "minItems": 1,
            "maxItems": 50,
            "items": {
                "type": "object",
                "additionalProperties": False,
                "required": ["label", "how_to", "expected_outcome"],
                "properties": {
                    "label": {"type": "string", "minLength": 1, "maxLength": 200},
                    "how_to": {"type": "string", "minLength": 1, "maxLength": 3000},
                    "expected_outcome": {"type": "string", "minLength": 1, "maxLength": 2000},
                },
            },
        },
        "metrics": {
            "type": "array",
            "minItems": 1,
            "maxItems": 30,
            "items": {"type": "string", "minLength": 1, "maxLength": 200},
        },
        "timeline_weeks": {
            "type": "array",
            "minItems": 1,
            "maxItems": 52,
            "items": {
                "type": "object",
                "additionalProperties": False,
                "required": ["week", "focus", "deliverable"],
                "properties": {
                    "week": {"type": "integer", "minimum": 1, "maximum": 104},
                    "focus": {"type": "string", "minLength": 1, "maxLength": 500},
                    "deliverable": {"type": "string", "minLength": 1, "maxLength": 1000},
                },
            },
        },
        "risks": {
            "type": "array",
            "minItems": 0,
            "maxItems": 30,
            "items": {
                "type": "object",
                "additionalProperties": False,
                "required": ["risk", "mitigation"],
                "properties": {
                    "risk": {"type": "string", "minLength": 1, "maxLength": 500},
                    "mitigation": {"type": "string", "minLength": 1, "maxLength": 1000},
                },
            },
        },
    },
}


def write_schema(schema_path: Path | None = None) -> Path:
    schema_path = schema_path or DATA_DIR / f"plan_schema_{datetime.now(UTC).strftime('%Y-%m-%d_%H%M')}.json"
    Draft202012Validator.check_schema(PLAN_SCHEMA)
    schema_path.parent.mkdir(parents=True, exist_ok=True)
    with schema_path.open("w", encoding="utf-8") as f:
        json.dump(PLAN_SCHEMA, f, ensure_ascii=False, indent=2)
    return schema_path


def validate_plan(plan: dict) -> None:
    try:
        Draft202012Validator(PLAN_SCHEMA).validate(plan)
    except ValidationError as e:
        raise ValueError(f"Schema validation failed: {e.message}") from e


## Prompt construction utilities (from `prompts.py`)
These helpers build the system/user prompts and the response format used by the generator scripts.


In [ ]:
# --- prompts helpers ---

PROMPT_VERSION = "PT-20251102-1"

SYSTEM_PROMPT = """
You are a senior computer scientist and mentor.
Transform a small set of computer science papers into a structured, actionable learning plan.

Hard requirements (read carefully):
- Output MUST be a single JSON object ONLY (no commentary, no markdown, no code fences).
- Follow the JSON schema fields EXACTLY:
  goal, study_level, source_papers, metadata, plan_overview, reading_order, actions, metrics, timeline_weeks, risks.
- Use ONLY the provided papers. Do NOT invent or cite any paper that is not in the provided list.
- All values in reading_order[].paper_id MUST be chosen from source_papers,
  and source_papers MUST include ALL provided paper IDs (no missing IDs, no invented IDs).
- Be concise and execution-oriented (clear steps, measurable outcomes).
- If information is insufficient, use fewer items and add a risk item explaining the limitation.
- Do NOT add any extra fields not defined by the schema.

Language and style:
- When referring to the learner, always address them directly as "You".
- Do NOT describe the learner in the third person (e.g., "a junior computer science student").

Schema guidance:
- goal: copy the user goal faithfully in meaning.
- study_level: choose from ["beginner","intermediate","advanced"]; if unsure, prefer "intermediate".
- source_papers: MUST contain all paper_id strings from the provided list (do not drop any, do not invent IDs).
- metadata:
  - prompt_version: provided by tooling.
  - model: provided by tooling.
  - created_at: current UTC in ISO8601 (e.g., 2025-11-01T12:34:56Z).
  - tokens_estimated: include only if values are provided; otherwise omit.
- plan_overview: 1-2 short paragraphs explaining rationale and overall strategy.
- reading_order: {paper_id, why_first, key_questions[]} with concrete technical questions.
- actions: 3-10 items of {label, how_to, expected_outcome}, focusing on reproducible tasks.
- metrics: 2-8 measurable indicators.
- timeline_weeks: 2-12 items; week starts at 1.
- risks: 0-8 items; realistic, technical risks only.

Output rule:
- Return ONLY the JSON object that conforms to the provided JSON schema.
""".strip()


def clip(text: str, max_chars: int) -> str:
    if not isinstance(text, str):
        return ""
    if len(text) <= max_chars:
        return text
    return text[: max_chars - 3] + "..."


def norm_str(x: Any) -> str:
    if x is None:
        return ""
    return str(x)


def load_plan_schema(schema_path: str | Path) -> Dict[str, Any]:
    with Path(schema_path).open("r", encoding="utf-8") as f:
        return json.load(f)


def build_response_format(schema: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "type": "json_schema",
        "json_schema": {
            "name": schema.get("title", "PaperTrailLearningPlan"),
            "schema": schema,
            "strict": True,
        },
    }


def build_user_prompt(
    goal: str,
    papers: List[Dict[str, Any]],
    *,
    audience: str = "you",
    study_level_hint: Optional[str] = None,
    model_id: str = "gpt-5",
    prompt_version: str = PROMPT_VERSION,
    max_abstract_chars: int = 1200,
) -> str:
    if study_level_hint not in {"beginner", "intermediate", "advanced", None}:
        study_level_hint = None
    if not goal:
        for p in papers:
            q = (p.get("query") or "").strip()
            if q:
                goal = q
                break

    lines: List[str] = []
    lines.append(f"Goal: {goal}")
    lines.append(f"Audience: {audience}")
    if study_level_hint:
        lines.append(f"StudyLevelHint: {study_level_hint}")
    lines.append("Selected research papers (id, title, authors, year, citation_count, categories, abstract):")
    allowed_ids = [norm_str(p.get("id")) for p in papers if p.get("id")]
    lines.append("AllowedPaperIDsJSON: " + json.dumps(allowed_ids, ensure_ascii=False))

    for idx, p in enumerate(papers, start=1):
        pid = norm_str(p.get("id"))
        title = norm_str(p.get("title"))
        authors = norm_str(p.get("authors"))
        year = norm_str(p.get("year"))
        cites = norm_str(p.get("citation_count"))
        cats = norm_str(p.get("categories"))
        abstract = clip(norm_str(p.get("abstract")), max_abstract_chars)

        lines.append(f"{idx}) ID: {pid}")
        lines.append(f"   Title: {title}")
        lines.append(f"   Authors: {authors}")
        lines.append(f"   Year: {year}")
        lines.append(f"   Citations: {cites}")
        lines.append(f"   Categories: {cats}")
        lines.append(f"   Abstract: {abstract}")

    lines.append(
        "Return a single JSON object that strictly conforms to the provided JSON schema. "
        "Set source_papers to contain ALL IDs listed in AllowedPaperIDsJSON (do not drop any, do not invent IDs). "
        "Design reading_order so that EVERY paper_id from source_papers appears at least once, "
        "and the sequence forms a coherent learning path for the user."
    )

    return "
".join(lines)


def build_messages(
    goal: str,
    papers: List[Dict[str, Any]],
    *,
    audience: str = "you",
    study_level_hint: Optional[str] = None,
    model_id: str = "gpt-4.1",
    prompt_version: str = PROMPT_VERSION,
    max_abstract_chars: int = 1200,
) -> List[Dict[str, str]]:
    user_prompt = build_user_prompt(
        goal,
        papers,
        audience=audience,
        study_level_hint=study_level_hint,
        model_id=model_id,
        prompt_version=prompt_version,
        max_abstract_chars=max_abstract_chars,
    )
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ]


## Plan generation (from `generate_plan_v3.py`)
Call the OpenAI Responses API with the prompts above, validate the structured JSON, and save artifacts/logs.


In [ ]:
# --- plan generation helpers ---

def read_papers_jsonl(path: Path) -> List[Dict[str, Any]]:
    papers = []
    with path.open("r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            s = line.strip()
            if not s:
                continue
            try:
                papers.append(json.loads(s))
            except json.JSONDecodeError as e:
                raise ValueError(f"bad json at line {i}: {e}") from e
    return papers


def pick_goal_from_query(papers: List[Dict[str, Any]]) -> str:
    for p in papers:
        q = (p.get("query") or "").strip()
        if q:
            return q
    raise ValueError("no 'query' found in input papers")


def assert_reading_order_from_source(plan: dict) -> None:
    src = set(plan.get("source_papers") or [])
    bad = [it.get("paper_id") for it in plan.get("reading_order", []) if it.get("paper_id") not in src]
    if bad:
        raise ValueError(f"reading_order contains IDs not in source_papers: {bad}")


def collect_topics_from_recommend(prefer_view: str = "default") -> Optional[str]:
    if prefer_view:
        preferred_file = RECOMMEND_DIR / f"recommend_{prefer_view}.json"
        if preferred_file.exists():
            recommend_file = preferred_file
        else:
            recommend_files = sorted(
                RECOMMEND_DIR.glob("recommend_*.json"),
                key=lambda p: p.stat().st_mtime,
                reverse=True,
            )
            if not recommend_files:
                return None
            recommend_file = recommend_files[0]
    else:
        recommend_files = sorted(
            RECOMMEND_DIR.glob("recommend_*.json"),
            key=lambda p: p.stat().st_mtime,
            reverse=True,
        )
        if not recommend_files:
            return None
        recommend_file = recommend_files[0]

    topics_set = set()
    try:
        with recommend_file.open("r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    rec = json.loads(line)
                    topics = rec.get("topics")
                    if topics and isinstance(topics, list):
                        topics_set.update(t for t in topics if t)
                except (json.JSONDecodeError, Exception):
                    continue
    except Exception:
        return None

    if not topics_set:
        return None

    topics_list = sorted(list(topics_set))
    topics_str = ", ".join(topics_list)

    return (
        "The selected papers may also cover research areas including: "
        f"{topics_str}. You may gain a broader understand of the field "
        "by exploring these related research directions"
    )


def inject_paper_titles(plan: dict, papers: List[Dict[str, Any]]) -> None:
    if not papers:
        return

    id_to_title = {p.get("id"): p.get("title", "") for p in papers if p.get("id")}

    reading_order = plan.get("reading_order")
    if not reading_order or not isinstance(reading_order, list):
        return

    for item in reading_order:
        if not isinstance(item, dict):
            continue
        pid = item.get("paper_id")
        if pid and pid in id_to_title:
            item["paper_title"] = id_to_title[pid]


def make_trace_id() -> str:
    ts = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")
    return f"{ts}-{uuid.uuid4().hex[:8]}"


def log_jsonl(path: Path, obj: dict) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "
")


def _normalize_text_format(fmt: Dict[str, Any]) -> Dict[str, Any]:
    if fmt.get("type") == "json_schema" and "schema" in fmt:
        if "name" not in fmt:
            fmt["name"] = "plan_schema"
        fmt.setdefault("strict", True)
        return fmt

    if fmt.get("type") == "json_schema" and isinstance(fmt.get("json_schema"), dict):
        js = fmt["json_schema"]
        name = js.get("name") or "plan_schema"
        strict = js.get("strict", True)
        schema = js.get("schema")
        if not isinstance(schema, dict):
            raise ValueError("text_format.json_schema.schema must be an object")
        return {
            "type": "json_schema",
            "name": name,
            "schema": schema,
            "strict": strict,
        }

    raise ValueError("Unsupported text_format shape for Responses API")


def call_with_retries(
    client: OpenAI,
    model: str,
    messages: List[Dict[str, str]],
    text_format: Dict[str, Any],
    temperature: float,
    timeout_sec: int,
    max_retries: int,
):
    inputs = [
        {
            "role": m["role"],
            "content": [
                {"type": "input_text", "text": m["content"]},
            ],
        }
        for m in messages
    ]

    last_err: Optional[Exception] = None

    for attempt in range(1, max_retries + 1):
        start = time.time()
        try:
            resp = client.responses.create(
                model=model,
                input=inputs,
                temperature=temperature,
                text={"format": text_format},
                timeout=timeout_sec,
            )
            latency = time.time() - start
            return resp, latency

        except Exception as e:
            last_err = e
            if attempt == max_retries:
                raise
            time.sleep(2 * attempt)

    raise last_err


def extract_text(resp) -> str:
    try:
        txt = getattr(resp, "output_text", None)
        if txt:
            return txt

        if hasattr(resp, "output") and resp.output:
            first = resp.output[0]
            content = getattr(first, "content", None)
            if content:
                part = content[0]
                part_text = getattr(part, "text", None)

                if isinstance(part_text, str):
                    return part_text

                if hasattr(part_text, "value"):
                    return part_text.value

                if isinstance(part_text, dict):
                    if "value" in part_text:
                        return str(part_text["value"])
                    if "text" in part_text:
                        return str(part_text["text"])

                if isinstance(part, dict) and "text" in part:
                    return str(part["text"])

        return ""
    except Exception:
        return ""


def validate_against_schema(schema: dict, data: dict) -> None:
    try:
        Draft202012Validator(schema).validate(data)
    except ValidationError as e:
        loc = " -> ".join([str(p) for p in e.path]) or "<root>"
        raise ValueError(f"schema fail at [{loc}]: {e.message}") from e


def estimate_cost_usd(model: str, in_tokens: Optional[int], out_tokens: Optional[int]) -> Optional[float]:
    try:
        in_price = float(os.environ.get("PRICE_IN_USD_PER_1K", "0"))
        out_price = float(os.environ.get("PRICE_OUT_USD_PER_1K", "0"))
        if in_tokens is None or out_tokens is None:
            return None
        return (in_tokens / 1000.0) * in_price + (out_tokens / 1000.0) * out_price
    except Exception:
        return None


def generate_plan(
    *,
    model: str = os.environ.get("OPENAI_MODEL", "gpt-4.1-mini"),
    temperature: float = float(os.environ.get("OPENAI_TEMPERATURE", "0.2")),
    timeout_sec: int = int(os.environ.get("OPENAI_TIMEOUT", "60")),
    max_retries: int = int(os.environ.get("OPENAI_MAX_RETRIES", "5")),
    schema_path: Optional[Path] = None,
    papers_path: Optional[Path] = None,
) -> dict:
    schema_path = schema_path or max(DATA_DIR.glob("plan_schema_*.json"), key=lambda f: f.stat().st_mtime)
    papers_path = papers_path or max(DATA_DIR.glob("standardize_input_*.json"), key=lambda f: f.stat().st_mtime)

    schema = load_plan_schema(str(schema_path))
    papers = read_papers_jsonl(papers_path)
    if not papers:
        raise ValueError("no papers")

    goal = pick_goal_from_query(papers)

    messages = build_messages(
        goal=goal,
        papers=papers,
        study_level_hint=None,
        max_abstract_chars=800,
    )
    text_format = _normalize_text_format(build_response_format(schema))

    client = OpenAI()
    trace_id = make_trace_id()

    resp, latency = call_with_retries(
        client=client,
        model=model,
        messages=messages,
        text_format=text_format,
        temperature=temperature,
        timeout_sec=timeout_sec,
        max_retries=max_retries,
    )

    text = extract_text(resp).strip()
    try:
        plan = json.loads(text)
    except json.JSONDecodeError as e:
        debug_path = DATA_DIR / "artifacts" / f"bad_output_{trace_id}.txt"
        debug_path.write_text(text, encoding="utf-8")
        raise ValueError(f"model did not return valid JSON (see {debug_path})") from e

    all_ids: List[str] = [p.get("id") for p in papers if p.get("id")]
    plan["source_papers"] = all_ids

    meta = plan.get("metadata") or {}
    meta["prompt_version"] = PROMPT_VERSION
    meta["model"] = model
    meta["created_at"] = datetime.now(timezone.utc).isoformat()
    plan["metadata"] = meta

    validate_against_schema(schema, plan)
    assert_reading_order_from_source(plan)

    inject_paper_titles(plan, papers)

    topics_suggestion = collect_topics_from_recommend(prefer_view="default")
    if topics_suggestion:
        plan["_topics_suggestion"] = topics_suggestion

    artifacts = DATA_DIR / "artifacts"
    artifacts.mkdir(parents=True, exist_ok=True)
    out_path = artifacts / f"plan_{trace_id}.json"
    out_path.write_text(json.dumps(plan, ensure_ascii=False, indent=2), encoding="utf-8")

    latest_path = DATA_DIR / "plan_latest.json"
    latest_path.write_text(json.dumps(plan, ensure_ascii=False, indent=2), encoding="utf-8")

    usage = getattr(resp, "usage", None)
    in_tokens = None
    out_tokens = None
    if usage is not None:
        in_tokens = getattr(usage, "input_tokens", None) or getattr(usage, "prompt_tokens", None)
        out_tokens = getattr(usage, "output_tokens", None) or getattr(usage, "completion_tokens", None)

    cost = estimate_cost_usd(model, in_tokens, out_tokens)

    log_obj = {
        "trace_id": trace_id,
        "ts_utc": datetime.now(timezone.utc).isoformat(),
        "prompt_version": PROMPT_VERSION,
        "model": model,
        "temperature": temperature,
        "latency_sec": round(latency, 3) if latency is not None else None,
        "tokens_in": in_tokens,
        "tokens_out": out_tokens,
        "cost_usd_estimate": cost,
        "artifact": str(out_path),
        "schema_file": str(schema_path),
        "papers_file": str(papers_path),
        "status": "ok",
    }

    log_dir = DATA_DIR / "logs"
    log_dir.mkdir(parents=True, exist_ok=True)
    log_path = log_dir / f"inference_{datetime.now(UTC).strftime('%Y-%m-%d_%H%M')}.json"
    log_jsonl(log_path, log_obj)

    print("
== Run Summary ==")
    print(f"trace_id: {trace_id}")
    print(f"model: {model}  temp: {temperature}")
    print(
        f"latency: {round(latency, 2) if latency is not None else 'n/a'}s  "
        f"tokens(in/out): {in_tokens}/{out_tokens}  cost~: {cost}"
    )
    print(f"artifact: {out_path}")
    print(f"latest:   {latest_path}")

    return plan


### Example usage
The cells above can be run individually. A minimal happy path is:
1. `build_selected_papers()` to create a fresh `standardize_input_*.json`.
2. `schema_path = write_schema()` to emit the latest schema.
3. `plan = generate_plan(schema_path=schema_path)` to call OpenAI and store artifacts/logs.

Make sure `OPENAI_API_KEY` (and optional pricing env vars) are set in the environment before running step 3.
